### Simple examples of ensembling models

Expected that hyperparameters of base models are already tuned

In [1]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

In [2]:
SHIFT = 190
os.environ['LIGHTGBM_EXEC'] = r'D:\LightGBM\LightGBM\windows\x64\Release\lightgbm.exe'  # path to LightGBM executable

In [3]:
df = pd.read_csv(r'D:\allstate\train.csv.zip', compression='zip')

for column in tqdm(df.columns):
    encoder = LabelEncoder()
    if column.startswith('cat') :
        df[column] = encoder.fit_transform(df[column])
        df['{}_sz'.format(column)] = df[column].map(df.groupby(column).size())

100%|████████████████████████████████████████████████████████████████████████████████| 132/132 [01:03<00:00,  2.06it/s]


In [4]:
df.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cat107_sz,cat108_sz,cat109_sz,cat110_sz,cat111_sz,cat112_sz,cat113_sz,cat114_sz,cat115_sz,cat116_sz
0,1,0,1,0,1,0,0,0,0,1,...,22405,21421,3142,4131,32401,17669,7033,131693,26813,3194
1,2,0,1,0,0,0,0,0,0,1,...,20236,42435,152918,3271,128395,7122,26191,131693,26813,9202
2,5,0,1,0,0,1,0,0,0,1,...,47310,9299,21933,1745,128395,2257,6079,131693,7090,2632
3,10,1,1,0,1,0,0,0,0,1,...,20236,42435,152918,24592,32401,8453,22030,131693,26813,20244
4,11,0,1,0,1,0,0,0,0,1,...,28560,65512,73,2681,32401,1351,26191,131693,43866,10162


###### For speed and simplicity there are only one split 80% / 20%
This is equivalent to a single iteration of 5-fold CV

In [5]:
train, test = train_test_split(df, test_size = 0.2, random_state = 0)
y_train, y_test = train['loss'], test['loss']
del train['loss'], test['loss']

###### Fit simple XGBoost and LightGBM models (without any tuning)

In [6]:
%%time
dtrain = xgb.DMatrix(train, label=np.log(y_train + SHIFT))
dtest = xgb.DMatrix(test)
params = {'silent': 1}
model = xgb.train(params, dtrain, 100)
y_pred_1 = np.exp(model.predict(dtest)) - SHIFT
print(mean_absolute_error(y_pred_1, y_test))

1162.57690233
Wall time: 1min 12s


In [7]:
%%time
from pylightgbm.models import GBMRegressor
params = {'verbose': False, 'num_iterations': 100}
clf = GBMRegressor(**params)
clf.fit(train, np.log(y_train + SHIFT))
y_pred_2 = np.exp(clf.predict(test)) - SHIFT
print(mean_absolute_error(y_pred_2, y_test))

1150.51258449
Wall time: 1min 51s


###### Get mean of this predictions

In [8]:
y_pred = (y_pred_1 + y_pred_2)/2
print(mean_absolute_error(y_pred, y_test))

1147.48434138


###### Try to choose weights for convex combination of predictions
i.e the sum of the weights is equal to one

In [9]:
best_a = None
min_err = 10**6
for a in np.arange(.05, 1, .05):
    y_pred = a*y_pred_1 + (1 - a)*y_pred_2
    err = mean_absolute_error(y_pred, y_test)
    if err < min_err:
        min_err = err
        best_a = a
    print(a, err)
print('----------------')
print('best:', best_a, min_err)

0.05 1149.34982513
0.1 1148.40174425
0.15 1147.62488821
0.2 1147.04778303
0.25 1146.6589999
0.3 1146.46163812
0.35 1146.45360075
0.4 1146.60493836
0.45 1146.94584922
0.5 1147.48434138
0.55 1148.19759288
0.6 1149.10182459
0.65 1150.15430338
0.7 1151.38349634
0.75 1152.79689637
0.8 1154.37413736
0.85 1156.14686535
0.9 1158.13413016
0.95 1160.27132575
----------------
best: 0.35 1146.45360075


The result is better than simple mean.

When using such method better models will have larger weight. This can be a problem if you have models with different quality.

### Stacking

###### Get out of fold predictions

In [10]:
import numpy as np
import xgboost as xgb
from pylightgbm.models import GBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

N_SPLITS = 5

def get_xgb_pred(X_train, X_test, y_train, params=None, n_trees=100, res_transform=None):
    if params is None:
        params = {'silent': 1}
    if res_transform is None:
        res_transform = lambda x: x
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test)
    model = xgb.train(params, dtrain, n_trees)
    return res_transform(model.predict(dtest))

def get_lightgbm_pred(X_train, X_test, y_train, params=None, n_trees=100, res_transform=None):
    if params is None:
        params = {'verbose': False}
    if not 'num_iterations' in params:
        params['num_iterations'] = n_trees
    if res_transform is None:
        res_transform = lambda x: x
    clf = GBMRegressor(**params)
    clf.fit(X_train, y_train)
    return res_transform(clf.predict(X_test))

def predict_outoffolds(X, y, X_test=None, func='get_xgb_pred', params=None, res_transform=None, n_splits=3):
    if X_test is None:
        kf = KFold(n_splits=n_splits)
        y_pred = np.zeros_like(y.values)
        for train_inds, test_inds in kf.split(X):
            y_pred[test_inds] = globals()[func](X.iloc[train_inds], X.iloc[test_inds],
                                                y.iloc[train_inds], res_transform=res_transform, params=params)
        return y_pred
    else:
        return globals()[func](X, X_test, y)

In [11]:
train_stack = pd.DataFrame(index=train.index)
test_stack = pd.DataFrame(index=test.index)

In [12]:
%%time
train_stack['y_xgb'] = predict_outoffolds(train, np.log(y_train + SHIFT), func='get_xgb_pred')
test_stack['y_xgb'] = predict_outoffolds(train, np.log(y_train + SHIFT), X_test=test, func='get_xgb_pred')

train_stack['y_lightgbm'] = predict_outoffolds(train, np.log(y_train + SHIFT), func='get_lightgbm_pred')
test_stack['y_lightgbm'] = predict_outoffolds(train, np.log(y_train + SHIFT), X_test=test, func='get_lightgbm_pred')

train_stack['y_xgb'] = np.exp(train_stack.y_xgb) - SHIFT
test_stack['y_xgb'] = np.exp(test_stack.y_xgb) - SHIFT

train_stack['y_lightgbm'] = np.exp(train_stack.y_lightgbm) - SHIFT
test_stack['y_lightgbm'] = np.exp(test_stack.y_lightgbm) - SHIFT

Wall time: 6min 2s


In [13]:
print(mean_absolute_error(train_stack.y_xgb.fillna(0), y_train),
      mean_absolute_error(train_stack.y_lightgbm.fillna(0), y_train),
      mean_absolute_error(test_stack.y_xgb.fillna(0), y_test),
      mean_absolute_error(test_stack.y_lightgbm.fillna(0), y_test))

1168.72249866 1149.18904792 1162.57690233 1150.51258449


###### Fit linear regression without bias on this two features

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
lr = LinearRegression(fit_intercept=False)
lr.fit(train_stack[['y_xgb', 'y_lightgbm']], y_train)
y_pred = lr.predict(test_stack[['y_xgb', 'y_lightgbm']])
print(mean_absolute_error(y_pred, y_test))

1166.85276723


In [15]:
print(*lr.coef_)

0.420327063731 0.693043542054


###### Fit nonlinear model on second level
concatenate meta features with original features

In [15]:
_train = pd.concat([train, train_stack], axis=1)
_test = pd.concat([test, test_stack], axis=1)

In [17]:
%%time
from pylightgbm.models import GBMRegressor
params = {'verbose': False, 'num_iterations': 100}
clf = GBMRegressor(**params)
clf.fit(_train, np.log(y_train + SHIFT))
y_pred = np.exp(clf.predict(_test)) - SHIFT
print(mean_absolute_error(y_pred, y_test))

1145.86787888
Wall time: 1min 4s


That's better than convex combination of predictions

###### Try to add yet another LightGBM
parameters are intentionally set to non-optimal values to get a weaker model

In [18]:
%%time
params = {
    'num_iterations': 100,
    'num_leaves': 127,
    'feature_fraction': .5,
    'bagging_fraction': .5,
    'learning_rate': .06,
    'min_data_in_leaf': 10,
    'verbose': False
}
from pylightgbm.models import GBMRegressor
clf = GBMRegressor(**params)
clf.fit(train, np.log(y_train + SHIFT))
y_pred_3 = np.exp(clf.predict(test)) - SHIFT
print(mean_absolute_error(y_pred_3, y_test))

1163.78679666
Wall time: 1min


In [19]:
y_pred = (y_pred_1 + y_pred_2 + y_pred_3)/3
print(mean_absolute_error(y_pred, y_test))

1148.27363456


weak model worsened the result of averaging

##### Convex combination of predictions tuned by Sequential Least Squares Programming solver
https://github.com/jdwittenauer/kaggle/blob/master/OttoGroup/find_ensemble_weights.py

http://stackoverflow.com/questions/35631192/element-wise-constraints-in-scipy-optimize-minimize

In [21]:
from scipy.optimize import minimize
def log_loss_func(weights):
    """
    scipy minimize will pass the weights as a numpy array
    """
    final_prediction = 0
    for weight, prediction in zip(weights, preds):
            final_prediction += weight*prediction
    return mean_absolute_error(y_test, final_prediction)

In [22]:
preds = [y_pred_1, y_pred_2, y_pred_3]

In [23]:
init_weights = np.ones(len(preds)) / len(preds)
# adding constraints
cons = ({'type': 'eq', 'fun': lambda w: 1-sum(w)})  # weights are sum to 1
bounds = [(0, 1)] * len(preds)  # and bounded between 0 and 1
w = minimize(log_loss_func, init_weights, method='SLSQP', bounds=bounds, constraints=cons)

In [24]:
w.x

array([ 0.3332174,  0.3332174,  0.3335652])

In [25]:
mean_absolute_error(np.dot(w.x.reshape(1, -1), np.vstack([y_pred_1, y_pred_2, y_pred_3])).ravel(), y_test)

1148.2759080673209

Slightly better than simple mean, but it is not the general case

###### Get out-of-fold prediction for this model

In [26]:
%%time
train_stack['y_lightgbm2'] = predict_outoffolds(train, np.log(y_train + SHIFT), func='get_lightgbm_pred', params=params)
test_stack['y_lightgbm2'] = predict_outoffolds(train, np.log(y_train + SHIFT), X_test=test, func='get_lightgbm_pred',
                                               params=params)

train_stack['y_lightgbm2'] = np.exp(train_stack.y_lightgbm2) - SHIFT
test_stack['y_lightgbm2'] = np.exp(test_stack.y_lightgbm2) - SHIFT

Wall time: 3min 25s


In [27]:
print(mean_absolute_error(train_stack.y_lightgbm2.fillna(0), y_train),
      mean_absolute_error(test_stack.y_lightgbm2.fillna(0), y_test))

1158.31610102 1150.51258449


###### Fit nonlinear model on second level
concatenate meta features with original features

In [31]:
_train = pd.concat([train, train_stack], axis=1)
_test = pd.concat([test, test_stack], axis=1)

In [32]:
%%time
params = {'verbose': False, 'num_iterations': 100, 'num_leaves': 63}
clf = GBMRegressor(**params)
clf.fit(_train, np.log(y_train + SHIFT), test_data=[(_test, np.log(y_test + SHIFT))])
y_pred = np.exp(clf.predict(_test)) - SHIFT
print(mean_absolute_error(y_pred, y_test))

1145.41550548
Wall time: 1min 8s


###### Some links
http://mlwave.com/kaggle-ensembling-guide/

https://www.kaggle.com/c/allstate-claims-severity/forums/t/25743/stacking-understanding-python-package-for-stacking

https://www.kaggle.com/c/allstate-claims-severity/forums/t/25240/weights-in-an-ensemble